# 2018-10-05 Batch effects

This is just a vague hope that there will be a way to avoid the removal of batch effects, just due to the fact that putting together the two batches and clustering will do the magic.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mybiotools as mbt
import os, sys, gzip
import pandas as pd
import umap

In [ ]:
# define paths and file names
sc_hiv_rootdir = '%s/work/CRG/projects/sc_hiv'%(os.getenv('HOME'))
datadir = '%s/data'%(sc_hiv_rootdir)
matrices_dir = '%s/matrices'%(datadir)

In [ ]:
class SsHIVExperiment :
    def __init__(self, sample_name) :
        
        # sample name
        self.sample_name = sample_name
        
        # expression matrix
        self.matrix_fname = '%s/%s.tsv.gz'%(matrices_dir, sample_name)
        # we use the read_csv function with option "index_col = 'gene'" so that we will
        # be able to classify the rows of the file according to the gene name
        self.matrix = pd.read_csv(self.matrix_fname,
                                  delimiter='\t',
                                  index_col='gene').transpose()

In [ ]:
# parse the experiment matrices and put everything in a Pandas DataFrame
sample_names = ['P2449', 'P2458']
experiments = []
for sample_name in sample_names :
    experiment = SsHIVExperiment(sample_name)
    experiment.labels = pd.Series(['Jurkat']*6 + ['J-Lat+DMSO']*30 + ['J-Lat+SAHA']*60,
                                  index=experiment.matrix.index)
    experiments.append(experiment)

In [ ]:
threshold = 100000.0
for experiment in experiments :
    experiment.alive_mask = experiment.matrix.sum(axis=1) > threshold
    experiment.alive = experiment.matrix.loc[experiment.alive_mask]
    experiment.alive_labels = experiment.labels[experiment.alive_mask]

In [ ]:
alldata = pd.concat((experiments[0].alive, experiments[1].alive))

In [ ]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(alldata)

In [ ]:
def scatter_with_label(xy, labels, label, color) :
    mask = labels==label
    plt.scatter(xy[mask,0], xy[mask,1], c=color, label=label)

In [ ]:
colors = {'Jurkat' : 'r',
          'J-Lat+DMSO' : 'b',
          'J-Lat+SAHA' : 'g'}

fig = plt.figure()
alive_labels = pd.concat((experiments[0].alive_labels, experiments[1].alive_labels))
scatter_with_label(embedding, alive_labels, 'Jurkat', colors['Jurkat'])
scatter_with_label(embedding, alive_labels, 'J-Lat+DMSO', colors['J-Lat+DMSO'])
scatter_with_label(embedding, alive_labels, 'J-Lat+SAHA', colors['J-Lat+SAHA'])
plt.legend()
plt.show()

So it doesn't do the magic. Still there are kinda weird results here.